Source: https://github.com/samwit/langchain-tutorials/blob/main/RAG/YT_Chat_your_PDFs_Langchain_Template_for_creating.ipynb

In [1]:
# Loads .env variables
%load_ext dotenv
%dotenv

## Basic Chat PDF

We'll use `CharacterTextSplitter` to split the document into chunks, then convert these chunks into embeddings with `OpenAIEmbeddings`, and finally store these embedding vectors with `FAISS` to perform similarity searches

**[OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html)**
* OpenAI embedding model, necessary when using other OpenAI models
* [OpenAI Embeddings docs](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)


**[FAISS (Facebook AI Similarity Search)](https://python.langchain.com/docs/integrations/vectorstores/faiss):**
* Library for efficient similarity search and clustering of dense vectors.
* It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM.
* It also contains supporting code for evaluation and parameter tuning.
* [FAISS article](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS 
from tqdm import tqdm

## Reading the PDF

In [3]:
# file_name = "Rodolitos_recifes_peixes.pdf"
# file_name = "Bianchi, 2021_Estimating global biomass fishing_Science.pdf"
file_name = "LivroIntroducaoCienciasDoMar.pdf"

doc_reader = PdfReader(f"./articles/{file_name}")
doc_reader

In [5]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(tqdm(doc_reader.pages)):
    text = page.extract_text()
    if text:
        raw_text += text

print(f"{i+1} pages merged into {len(raw_text)} characters")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 603/603 [00:50<00:00, 12.00it/s]

603 pages merged into 1158200 characters


In [6]:
print(raw_text[:1000])

INTRODUÇÃO
ÀS CIÊNCIAS DO MAR
PPG-Mar
1ª REIMPRESSÃOFoto: Projeto Amazônia AzulJORGE  P. C ASTELLO  E LUIZ C. K RUG
ORGANIZADORES
Carla Rosana de Castro Aseff
Danilo Calazans
Débora Copstein Cuchiara
Elisa Helena Leão Fernandes
Frederico Pereira Brandini
Gilberto Henrique Griep
Iran Carlos Stalliviere Corrêa
Jair Weschenfelder
Jean Louis Valentin
José Angel Alvarez Perez
José Henrique Muelbert
Joseph Harari
Lauro Julio CalliariLúcio Franco de Sá Fernandes
Luis Felipe Hax Niencheski
Marcus Polette
Milton Lafourcade Asmus
Moysés Gonzalez Tessler
Osmar Olinto Möller Jr.
Paulo Cesar Oliveira Vergne de Abreu
Paulo da Cunha Lana
Regina Rodrigues
Ricardo de Camargo
Ronaldo Oliveira Cavalli
Xavier CastelloCOLABORADORES
EDITORA TEXTOS
Pelotas, 2017INTRODUÇÃO
ÀS CIÊNCIAS DO MAR
REVISOR  CIENTÍFICO
Danilo Calazans
ILUSTRAÇÕES
Kely Martinato© Copyright  Jorge P. Castello e Luiz C. Krug, 2017
EDITORA  TEXTOS
www.editoratextos.com.br
E-mail: contato@editoratextos.com.br
Fone: (53) 99143-8460
Pelotas

## Text Splitter

Takes the full text and splits it into chunks, for indexing. The chunk size is measured in characters, not tokens

In [7]:
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, # text overlap between chunks
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

len(texts)

Created a chunk of size 1076, which is longer than the specified 1000
Created a chunk of size 1084, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1178, which is longer than the specified 1000
Created a chunk of size 1506, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000
Created a chunk of size 1289, which is longer than the specified 1000
Created a chunk of size 1155, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000
Created a chunk of size 1189, which is longer than the specified 1000
Created a chunk of s

1506

In [8]:
print(texts[101])

O conhecimento da topografia submarina é fundamental para o geólogo marinho,
uma vez que lhe dará informações sobre a estrutura, a história tectônica, deposicional,erosiva e mesmo a origem das bacias oceânicas.
2  M ARGEM  CONTINENTAL
As margens continentais englobam as províncias fisiográficas que se associam à
passagem dos continentes para os oceanos. Com base em critérios tectônicos, é possívelreconhecer dois tipos de margens continentais: tipo Atlântico (ou passiva), comcomportamento tectônico estável por um longo período de tempo; e tipo Pacífico(ou ativa), com tectonismo ativo durante os últimos períodos geológicos.
As margens continentais do tipo Atlântico são características das margens dos
Oceanos Atlântico (com exceção das Antilhas), Índico (com exceção do arco de
Foto: Projeto Amazônia Azul46 IRAN CARLOS  STALLIVIERE  CORRÊA  E JAIR WESCHENFELDER
JORGE  P. C ASTELLO  E LUIZ C. K RUG  (O RG.)Sunda), Antártico (com exceção do arco de Scotia), Ártico e do mar da Noruega.


## Making the embeddings

In [9]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [11]:
# Creates a FAISS vector store using the article chunks and the OpenAI embeddding
docsearch = FAISS.from_texts(texts, embeddings)

In [12]:
query = "Talude continental"
docs = docsearch.similarity_search(query)

len(docs)

4

In [13]:
docs

[Document(page_content='fundos abissais de área de clima tropical/equatorial (Brasil).\n[Fonte: T ESSLER  e MAHIQUES , 2000].\n56 IRAN CARLOS  STALLIVIERE  CORRÊA  E JAIR WESCHENFELDER\nJORGE  P. C ASTELLO  E LUIZ C. K RUG  (O RG.)4  T ALUDE  CONTINENTAL\nO talude continental corresponde à parte da margem continental que apresenta\nmergulho acentuado (1:40 – 25 m/km) e que se estende a partir da borda da plataforma\ncontinental (F IG. 3.11). O topo do talude continental é marcado por uma mudança', metadata={}),
 Document(page_content="mergulho acentuado (1:40 – 25 m/km) e que se estende a partir da borda da plataforma\ncontinental (F IG. 3.11). O topo do talude continental é marcado por uma mudança\nbrusca no declive (quebra da plataforma), enquanto o pé do talude (talude inferior)possui declive mais suave, passando gradativamente para a elevação continental.Proporcionalmente, o talude continental constitui a porção mais íngreme do pisomarinho. Apresenta sensíveis variações de gradient

## Plain QA Chain

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [15]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # default model is being deprecated and is more expensive
    temperature=0
)

chain = load_qa_chain(
    llm, 
    chain_type="stuff"
)

In [16]:
# check the default LLM chain prompt
chain.llm_chain.prompt

ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [17]:
# query = "What is the article about?"
query = "\nO que são plataformas de zonas intermediárias?"

docs = docsearch.similarity_search(query)
answer = chain.run(input_documents=docs, question=query)
print(f"{query}\n")
print(f"{answer}\n")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..



O que são plataformas de zonas intermediárias?

Plataformas de zonas intermediárias são encontradas entre as latitudes de 55º e 45º. Elas apresentam uma maior importância das formações fluviais, colmatação dos lutitos, pelitos e sedimentos finos eólicos (loess), retrabalhamento de areias, dunas submarinas longitudinais formadas pelas correntes de marés, superfícies rochosas do substrato erodidas pelas correntes e desenvolvimento de depósitos de vasas. Além disso, podem apresentar formas paleocársticas nas superfícies.



In [18]:
query = "\nSumarize principais características do Talude Continental"
docs = docsearch.similarity_search(query)

# Ask ChatGPT the question sending the most similar pages as input_documents
answer = chain.run(input_documents=docs, question=query)
print(f"{query}\n")
print(f"{answer}\n")

# Ask ChatGPT the question without sending the input_documents
answer = chain.run(input_documents=[], question=query)
print(f"{query}\n")
print(f"{answer}\n")


Sumarize principais características do Talude Continental

O talude continental é a parte da margem continental que apresenta um mergulho acentuado, com declives que variam de 1:2 até 1:40. Sua inclinação média é de 4°17' e sua largura pode variar de pouco mais de 10 km até 200 km. O talude é bastante dissecado, com inúmeras irregularidades topográficas, como cânions submarinos, cristas e saliências rochosas, ravinas, terraços e escarpas. A maioria dos terraços e escarpas do talude são resultado de deslizamento de sedimentos, afloramentos de rochas ou falhamentos. O talude continental pode atingir profundidades de 100-200 m até 1.300-3.200 m, e em áreas onde se estendem pelas fossas oceânicas, pode chegar a 8.200 m.


Sumarize principais características do Talude Continental

O Talude Continental é uma área submarina que se estende desde a plataforma continental até as grandes profundidades oceânicas. Suas principais características incluem uma inclinação acentuada, geralmente entre 3

## Hugging Face model